In [ ]:


from utils import path


In [54]:
import os
import requests
import re
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import finnhub
from dotenv import load_dotenv
from pathlib import Path    
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import sys
sys.path.append('../') # Change the python path at runtime

# Self-created modules
from src.utils import path



# Set up Variables

In [16]:
load_dotenv()

SUBJECTIVITY_API_KEY = os.environ.get('SUBJECTIVITY_API_KEY')
POLARITY_API_KEY = os.environ.get('POLARITY_API_KEY')
INTENSITY_API_KEY = os.environ.get('INTENSITY_API_KEY')
FINNHUB_API_KEY = os.environ.get('FINNHUB_API_KEY')

In [33]:
BACKTEST_START_DATE = '2023-11-01'
BACKTEST_END_DATE = '2024-01-31'

# Finnhub API

- Finnhub API
https://finnhub.io/docs/api/company-news

# Fetch News Data

In [34]:
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# print(finnhub_client.general_news('general', min_id=0))

# Need to use _from instead of from to avoid conflict

# news_resp = finnhub_client.company_news('BA', 
#                                         _from=BACKTEST_START_DATE, 
#                                         to=BACKTEST_END_DATE)

# TODO: Explore
# Company Peers
# print(finnhub_client.company_peers('AAPL'))

In [63]:
cur_dir = Path.cwd()
root_dir = path.get_root_dir(cur_dir)
news_path = Path.joinpath(root_dir, 'data', 'raw', 'boeing_news_20231101_to_20240131.csv')
news_df = pd.DataFrame(pd.read_csv(news_path))
display(news_df)



,Unnamed: 0,category,datetime,headline,id,image,related,source,summary,url
0,0,company,1706743226,UPDATE 1-US FAA says Boeing 737 audit will rev...,125468439,https://s.yimg.com/cv/apiv2/social/images/yaho...,BA,Yahoo,The Federal Aviation Administration said on We...,https://finnhub.io/api/news?id=3c9f0ede6c5e041...
1,1,company,1706741163,"ROSEN, TRUSTED INVESTOR COUNSEL, Encourages Th...",125443555,NaN,BA,Finnhub,"NEW YORK, Jan. 31, 2024 /PRNewswire/ -- WHY: ...",https://finnhub.io/api/news?id=ca83085b0a7e26b...
2,2,company,1706740216,Boeing (BA) Q4 2023 Earnings Call Transcript,125468440,https://g.foolcdn.com/editorial/images/1/featu...,BA,Yahoo,BA earnings call for the period ending Decembe...,https://finnhub.io/api/news?id=48410f9fbdcd990...
3,3,company,1706738780,US FAA says Boeing 737 audit will review Spiri...,125468441,https://media.zenfs.com/en/reuters-finance.com...,BA,Yahoo,WASHINGTON (Reuters) -The Federal Aviation Adm...,https://finnhub.io/api/news?id=9b6ab2295db1269...
4,4,company,1706738746,Boeing CEO says company has 'much to prove',125468442,https://s.yimg.com/ny/api/res/1.2/oFjcSDkfl1nj...,BA,Yahoo,Boeing chief executive Dave Calhoun told emplo...,https://finnhub.io/api/news?id=fe1736dabb96447...
...,...,...,...,...,...,...,...,...,...,...
208,208,company,1706514420,European Airline Ryanair's Q3 Profit Slumps As...,125385770,NaN,BA,Benzinga,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=4808ba7cd9ad5bb...
209,209,company,1706513760,Alaska Airlines plane said to have left Boeing...,125384252,NaN,BA,Seeking Alpha,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=cae0cb6eed1d96a...
210,210,company,1706512579,News Highlights : Top Company News of the Day ...,125373537,NaN,BA,Finnhub,Evergrande Was Once China's Biggest Property D...,https://finnhub.io/api/news?id=8d715bff70ed539...
211,211,company,1706512440,Ryanair narrows profit guidance following clas...,125388237,https://images.mktw.net/im-88479800/social,BA,MarketWatch,Various online travel agents stopped listing R...,https://finnhub.io/api/news?id=762cc1080f80ec4...


In [44]:

# news_df.to_csv('boeing_news_20231101_to_20240131.csv')

In [67]:
# General information
print(news_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  213 non-null    int64 
 1   category    213 non-null    object
 2   datetime    213 non-null    int64 
 3   headline    213 non-null    object
 4   id          213 non-null    int64 
 5   image       89 non-null     object
 6   related     213 non-null    object
 7   source      213 non-null    object
 8   summary     211 non-null    object
 9   url         213 non-null    object
dtypes: int64(3), object(7)
memory usage: 16.8+ KB
None


In [71]:
# Check NA values
print(news_df.isna().sum())

# Convert UNIX to pandas datetime
news_df['datetime2'] = pd.to_datetime(news_df['datetime'], unit="s")
news_df.loc[pd.isna(news_df['summary']), :]


Unnamed: 0      0
category        0
datetime        0
headline        0
id              0
image         124
related         0
source          0
summary         2
url             0
datetime2       0
dtype: int64


,Unnamed: 0,category,datetime,headline,id,image,related,source,summary,url,datetime2
28,28,company,1706712966,The Boeing Company (BA) Q4 2023 Earnings Call ...,125437719,NaN,BA,SeekingAlpha,NaN,https://finnhub.io/api/news?id=97b3ae56c6a4ba0...,2024-01-31 14:56:06
143,143,company,1706586439,Boeing Customer Gol Files For Bankruptcy,125397543,NaN,BA,SeekingAlpha,NaN,https://finnhub.io/api/news?id=7f639fa386d7272...,2024-01-30 03:47:19


In [ ]:
news_df2 = news_df.drop(columns=['Unnamed: 0', 'image'])
news_df2.dropna(axis=1,)
news_df2

In [66]:
print(news_df['source'].unique())
print(news_df['id'].duplicated().sum())
for column in ['category', 'source']:
    print(f"Unique values in '{column}':")
    print(news_df[column].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  213 non-null    int64 
 1   category    213 non-null    object
 2   datetime    213 non-null    int64 
 3   headline    213 non-null    object
 4   id          213 non-null    int64 
 5   image       89 non-null     object
 6   related     213 non-null    object
 7   source      213 non-null    object
 8   summary     211 non-null    object
 9   url         213 non-null    object
dtypes: int64(3), object(7)
memory usage: 16.8+ KB
None
Unnamed: 0      0
category        0
datetime        0
headline        0
id              0
image         124
related         0
source          0
summary         2
url             0
dtype: int64
['Yahoo' 'Finnhub' 'MarketWatch' 'SeekingAlpha' 'Alliance News' 'TipRanks'
 'Benzinga' 'Seeking Alpha' 'Thefly.com' 'TalkMarkets' '247WallSt'
 'InvestorPlace' 'United

TODO: timezone
UNIX timezone 1706743226
https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjMxeO_w5CEAxWByjgGHQwwCT8QFnoECA0QAw&url=https%3A%2F%2Fnote.nkmk.me%2Fen%2Fpython-unix-time-datetime%2F%23%3A~%3Atext%3DUnix%2520time%2520is%2520the%2520number%2CPosix%2520time%252C%2520and%2520so%2520on.&usg=AOvVaw0s2rDBPUsnO1N5cO74o2AR&opi=89978449

# Parse

In [46]:
url = 'https://finnhub.io/api/news?id=3c9f0ede6c5e04181019b4ee12a18bcf1094b8970cfdff1f8d454a888134b2a2'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# The method to find the news content in the HTML will vary by site
# For example, if the news content is within a <p> tag
news_content = ' '.join(p.get_text() for p in soup.find_all('p'))
news_content

'(Adds background, comments, more from FAA) By David Shepardson WASHINGTON, Jan 31 (Reuters) - The Federal Aviation Administration said on Wednesday that its previously announced audit of Boeing 737 MAX manufacturing will look at all elements of production at Boeing and fuselage production at Spirit AeroSystems\'. The FAA said a team of two dozen aviation safety inspectors will conduct the audit at the Boeing 737 facility in Renton, Washington, and at Spirit in Wichita, Kansas. The audit will also examine how Boeing transfers unfinished work from suppliers to its production lines. The FAA announced the audit on Jan. 13 after the agency grounded 171 MAX 9 planes due to a mid-air cabin panel blowout on a new Alaska Airlines MAX 9. The FAA last week allowed the planes to fly again. The FAA said it is conducting enhanced oversight of Boeing and "will regularly assess trends, corrective actions and the effects of any changes to the quality system." The FAA did not say how long the greater s

- Notes from prof
    - here is a sample use for English: sentic.net/api/en/KEY.py?text=senticnet+is+pretty+cool

    - input text does not require any special formatting so feel free to use spaces instead of '+' or '%20'

    - ampersand, hashtag, semicolons, and braces ('&', '#', ';', '{', '}'), however, are illegal characters

    - hence, they should be replaced with colons (':') or removed entirely in the preprocessing phase

    - please note that:

    - 1) API keys are case-sensitive

    - 2) API keys will be valid for about one month

    - 3) API keys are personal and confidential

    - do not share nor use them from different devices or IP addresses

    - or else they will get terminated earlier

    - the capacity limit for our server is 8000 characters 

    - so our recommendation is to cap your input at about 1000 words

    - if you need to process bigger texts, you will have to split them into smaller parts

    - this is also a good idea in case you want to perform a finer-grained analysis of your input

    - all APIs, in fact, are designed to give you an overall judgement about the whole input

    - for more details, split your text into paragraphs or sentences and feed them to the API one by one

In [ ]:
news_txt = """
The latest black eye for Boeing's top-selling 737 MAX aircraft occurred Friday when a cabin panel blowout forced an Alaska Airlines (ALK.N) flight to make an emergency landing.
U.S. regulators ordered a temporary grounding of certain planes for safety checks. That's not as severe a move as the grounding of all MAX-family jets worldwide nearly five years ago following a pair of fatal crashes.

Here is a timeline of recent issues surrounding Boeing's (BA.N) MAX planes:
Advertisement · Scroll to continue
OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.

NOVEMBER 2018: The FAA and Boeing say they are evaluating the need for software or design changes to 737 MAX jets following the Lion Air crash.

MARCH 2019: An Ethiopian Airlines MAX crashes, killing all 157 people on board. China's aviation regulator becomes the first in the world to ground the MAX, followed by others including the U.S. Federal Aviation Administration.

Advertisement · Scroll to continue
APRIL 2019: The FAA forms an international team to review the safety of the 737 MAX. Boeing cuts monthly production by nearly 20%.

JULY 2019: Boeing posts its largest-ever quarterly loss.

SEPTEMBER 2019: Boeing's board of directors creates a permanent safety committee to oversee development, manufacturing and operation of its aircraft.

OCTOBER 2019: Boeing fires Kevin McAllister, the top executive of its commercial airplanes division.

Advertisement · Scroll to continue
DECEMBER 2020: The company fires CEO Dennis Muilenburg in the wake of the twin crashes.

JANUARY 2020: Boeing suspends 737 production, its biggest assembly-line halt in more than 20 years.

MAY 2020: Boeing resumes 737 MAX production at a "low rate."

JUNE 2020: Boeing begins a series of long-delayed flight tests of its redesigned 737 MAX with regulators at the controls.

SEPTEMBER 2020: An 18-month investigation by a U.S. House of Representatives panel finds Boeing failed in its design and development of the MAX as well as its transparency with the FAA, and that the FAA failed in oversight and certification.

NOVEMBER 2020: The U.S. FAA lifts the grounding order, allowing the 737 MAX to fly again.

DECEMBER 2020: Congress passes legislation to reform how the FAA certifies new airplanes, including requiring manufacturers to disclose certain safety-critical information to the FAA.

JANUARY 2021: The European Union Aviation Safety Agency approves the MAX's return to service in Europe.

MARCH 2021: China's aviation regulator says major safety concerns with the MAX needed to be "properly addressed" before conducting flight tests.

APRIL 2021: Boeing halts 737 MAX deliveries after electrical problems re-ground part of the fleet.
NOVEMBER 2021: Current and former Boeing company directors reach a $237.5 million settlement with shareholders to settle lawsuits over safety oversight of the 737 MAX.

OCTOBER 2022: The FAA tells Boeing that some key documents submitted as part of the certification review of the 737 MAX 7 are incomplete and others need a reassessment.

DECEMBER 2022: Congress agrees to extend a deadline for new standards for modern cockpit alerts stemming from the 2020 legislation after intense lobbying from Boeing.

APRIL 2023: Boeing pauses deliveries of some 737 MAXs to deal with a new supplier quality problem involving noncompliant fittings.
JULY 2023: Boeing's first delivery of the 737 MAX 7 is delayed to 2024.

AUGUST 2023: Boeing identifies a new 737 MAX supplier quality problem involving improperly drilled holes on the aft pressure bulkhead.

SEPTEMBER 2023: Boeing 737 MAX deliveries fall to their lowest levels since August 2021.
DECEMBER 2023: Boeing makes its first direct delivery of a 787 Dreamliner to China since 2019, seen as a precursor to China potentially unfreezing deliveries of the 737 MAX.
JANUARY 2024: An Alaskan Air flight is forced to conduct an emergency landing after a cabin panel blowout on a brand-new 737 MAX 9 plane. The U.S. FAA grounds certain 737 MAX 9 aircraft for safety checks.
"""

char_limit = 8000
# Remove invalid characters for API
news_txt2 = re.sub(r'[&#;{}]', '', news_txt)

# Remove advertisement-like texts
news_txt3 = re.sub(r'Advertisement · Scroll to continue', '', news_txt2)
display(news_txt3)

# paragraph.strip() does not include '' in the new list
# Removes leading and trailing white spaces
passages2 = [paragraph.strip() for paragraph in news_txt3.split('\n') if paragraph.strip()]

# Split based on 8000 characters
# passages = [news_txt3[i: i + char_limit] for i in range(0, len(news_txt3), char_limit)]

passages2

'\nThe latest black eye for Boeing\'s top-selling 737 MAX aircraft occurred Friday when a cabin panel blowout forced an Alaska Airlines (ALK.N) flight to make an emergency landing.\nU.S. regulators ordered a temporary grounding of certain planes for safety checks. That\'s not as severe a move as the grounding of all MAX-family jets worldwide nearly five years ago following a pair of fatal crashes.\n\nHere is a timeline of recent issues surrounding Boeing\'s (BA.N) MAX planes:\n\nOCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.\n\nNOVEMBER 2018: The FAA and Boeing say they are evaluating the need for software or design changes to 737 MAX jets following the Lion Air crash.\n\nMARCH 2019: An Ethiopian Airlines MAX crashes, killing all 157 people on board. China\'s aviation regulator becomes the first in the world to ground the MAX, followed by others including the U.S. Federal Aviation Administration.\n\n\nAPRIL 2019: The FAA forms an international

["The latest black eye for Boeing's top-selling 737 MAX aircraft occurred Friday when a cabin panel blowout forced an Alaska Airlines (ALK.N) flight to make an emergency landing.",
 "U.S. regulators ordered a temporary grounding of certain planes for safety checks. That's not as severe a move as the grounding of all MAX-family jets worldwide nearly five years ago following a pair of fatal crashes.",
 "Here is a timeline of recent issues surrounding Boeing's (BA.N) MAX planes:",
 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.',
 'NOVEMBER 2018: The FAA and Boeing say they are evaluating the need for software or design changes to 737 MAX jets following the Lion Air crash.',
 "MARCH 2019: An Ethiopian Airlines MAX crashes, killing all 157 people on board. China's aviation regulator becomes the first in the world to ground the MAX, followed by others including the U.S. Federal Aviation Administration.",
 'APRIL 2019: The FAA forms an internationa

# Sentiment analysis
- SenticNet
- TextBlob
- nltk
- BERT

TODO:
- Check whether need to unify the results
- Not included subjectivity yet

In [ ]:
# number of tweets pass through API

def analyse(text, LANG, KEY):
    APIURL = 'https://sentic.net/api/' + LANG + '/' + KEY + '.py?text='
    
    # label is the concept, polarity that is returned from API
    label = str(requests.get(APIURL + text).content)[2:-3]
    return label

res1 = analyse(passages2[3], 'en', POLARITY_API_KEY)
res2 = analyse(passages2[5], 'en', INTENSITY_API_KEY)
res3 = analyse(passages2[8], 'en', SUBJECTIVITY_API_KEY)

display(res1, res2, res3)



'POSITIVE'

'55'

'SUBJECTIVE'

In [ ]:
sentence = 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.'

blob = TextBlob(sentence)
polarity = blob.sentiment.polarity
subjectivity = blob.sentiment.subjectivity

print(f"Polarity: {polarity}")
print(f"Subjectivity: {subjectivity}")

Polarity: 0.0
Subjectivity: 0.0


In [ ]:


# nltk.download('vader_lexicon')  # Download the VADER lexicon if not already downloaded

sentence = 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.'

sid = SentimentIntensityAnalyzer()
sentiment_scores = sid.polarity_scores(sentence)

print(sentiment_scores)

# VADER compound score: https://stackoverflow.com/questions/40325980/how-is-the-vader-compound-polarity-score-calculated-in-python-nltk

{'neg': 0.239, 'neu': 0.761, 'pos': 0.0, 'compound': -0.6597}


In [ ]:
from transformers import pipeline

# Load the sentiment analysis pipeline with the chosen BERT model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_analysis = pipeline("sentiment-analysis", model=model_name)

# Example sentence
sentence = 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.'

# Get sentiment prediction
result = sentiment_analysis(sentence)

# Print the result
print(result)

# label is 1 to 5 stars. 1 is very negative, 5 is very positive
# Score is the confidence, between 0 to 1

config.json: 100%|██████████| 953/953 [00:00<00:00, 1.35MB/s]
pytorch_model.bin: 100%|██████████| 669M/669M [00:11<00:00, 59.6MB/s] 
/opt/homebrew/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
tokenizer_config.json: 100%|██████████| 39.0/39.0 [00:00<00:00, 877B/s]
vocab.txt: 100%|██████████| 872k/872k [00:00<00:00, 1.54MB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 284kB/s]


[{'label': '1 star', 'score': 0.8441336154937744}]
